In [1]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import keras.layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Concatenate, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.merge import concatenate

Using TensorFlow backend.


### Introdução

Deep Learning atualmente é uma abordagem considerada estado da arte para muitas aplicações, principalmente aquelas envolvendo classificação de imagens devido ao desafio ImageNet, que resultou em arquiteturas complexas de Deep Learning como AlexNet, VGG-16, GoogleLeNet e ResNet. O gráfico abaixo ilustra a performance dessas arquiteturas utilizando a base do ImageNet, em que, a arquitetura ResNet possui resultados melhores que o humano.

![alt text](imgs/imagenet.png "Title")

Essas arquiteturas podem ser implementadas utilizando a API Keras a partir de modelos não sequenciais. Assim, esse notebook ilustra uma implementação de uma rede com blocos da arquitetura Inceptionv4 com blocos residuais. A partir dessa implementação, é possível implementar arquiteturas diferenciadas de Deep Learning para resolver diversos problemas. 

### Inception v4

Uma arquitetura Inception possui uma estrutura dividida em dois blocos principais: stem e reduction. Stem é o módulo base da arquitetura, formado por uma sequência de redes convolucionais e pooling, os módulos reduction, por outro lado, são módulos em que cada camada é constituída por uma concatenação de convoluções e pooling. 

Além desses dois blocos principais, a Inception possui também uma versão com blocos residuais, conforme Figura abaixo. Esses blocos aumentam consideravelmente a performance da rede e aceleram também o treinamento. 

![alt text](imgs/architecture.png "Title")

Baseado nisso, as seções desse notebook são organizadas da seguinte forma:
* Stem: ilustra a implementação do bloco stem darede
* Reduction-A: ilustra a implementação do primeiro bloco reduction da arquitetura
* Resnet-A: ilustra a implementação do primeiro bloco residual 
* Reduction-B: ilustra a implementação do segundo bloco reduction
* Resnet-B: ilustra a implementação do segundo bloco residual
* Reduction-C: ilustra a implementação do terceiro bloco reduction
* Construindo a arquitetura Inceptionv4: ilustra o dataflow da arquitetura e realiza o treinamento


### Stem

O bloco stem é formado inicialmente por três camadas convolucionais seguidas, cada uma, por normalização do batch, sendo a primeira delas uma camada com 32 filtros de dimensão 3x3 e stride 2x2 e sem padding (valid). Uma vez que o modelo não é sequencial, a implementação do Keras não é definida como model=Sequential() e não possui o método model.add(camada). Ao invés disso, atribuiu-se a cada camada uma variável, passando-se como parâmetro dessa camada a camada anterior.

Por exemplo, para a primeira camada, chamamos o método do Keras Conv2D, atribuindo seu retorno a variável model. Além disso, para definir a entrada, após o Conv2D colocamos entre parentese (input_img), que representa a entrada da rede de dimensões 32x32x3. Em seguida, chamamos o método BatchNormalization atribuindo seu resultado novamente a variável model.

Após a implementação dessas três primeiras camadas, tem-se uma camada formada pela concatenação de uma camada convolucional com uma camada pooling. Para isso, definimos duas variáveis auxiliares (branch0 e branch1) e atribuimos a cada uma as respectivas camadas convolucionais e pooling. Após isso, utiliza-se o método concatenate do Keras para formar a camada da rede. Da mesma forma, podemos definir as demais camadas do Stem até a última concatenação, retornando-se o modelo final.

![alt text](imgs/stem.png "Title")



In [2]:
def stem(input_img):
    model = Conv2D(32, (3,3), strides=(2,2), padding='valid', activation='relu',name='stem')(input_img)
    model = BatchNormalization()(model)
    model = Conv2D(32, (3,3), padding='valid', activation='relu', name = 'conv1')(model)
    model = BatchNormalization()(model)
    model = Conv2D(64, (3,3), padding='same', activation='relu', name = 'conv2')(model)
    model = BatchNormalization()(model)
    branch0 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(model)
    branch1 = Conv2D(96, (3,3), strides=(2,2), padding='valid', activation='relu', name = 'conv3_branch1')(model)
    branch1 = BatchNormalization()(branch1)
    model = concatenate([branch0, branch1], axis=-1)
    branch0 = Conv2D(64, (1,1), padding='same', activation='relu', name = 'conv5_branch0')(model)
    branch0 = BatchNormalization()(branch0)
    branch0 = Conv2D(96, (3,3), padding='valid', activation='relu', name='conv6_branch0')(branch0)
    branch0 = BatchNormalization()(branch0)
    branch1 = Conv2D(64, (1,1), padding='same', activation='relu',name='conv7_branch1')(model)
    branch1 = Conv2D(64, (7,1), padding='same', activation='relu',name='conv8_branch1')(branch1)
    branch1 = BatchNormalization()(branch1)
    branch1 = Conv2D(64, (1,7), padding='same', activation='relu',name='conv9_branch1')(branch1)
    branch1 = BatchNormalization()(branch1)
    branch1 = Conv2D(96, (3,3), padding='valid', activation='relu',name='conv10_branch1')(branch1)
    branch1 = BatchNormalization()(branch1)
    model = concatenate([branch0, branch1], axis=-1)
    branch0 = Conv2D(192, (3,3), padding='valid', activation='relu', name = 'conv11_branch0')(model)
    branch0 = BatchNormalization()(branch0)
    branch1 = MaxPooling2D()(model)
    model = concatenate([branch0, branch1], axis=-1)
    return model

### Resnet A

Para implementação dos blocos residuais, inicialmente cria-se três branches (branch0,branch1 e branch2) e concatenamos o resultado das três para formação da camada na rede. Feito isso, essa camada é utilizada como entrada para uma camada convolucional com 384 filtros.

Por fim, para fazer um modelo residual, utiliza-se a função add do Keras passando-se como parâmetro o modelo resultante do bloco anterior (stem) e o modelo atual da camada (x). O resultado disso é o modelo de saída desse bloco.

![alt text](imgs/resnetA.png "Title")



In [3]:
def inception_resnet_A(model):
    branch0 = Conv2D(32, (1,1), padding='same', activation='relu',name='resnet_a_conv1_branch0')(model)
    branch0 = BatchNormalization()(branch0)
    branch1 = Conv2D(32, (1,1), padding='same', activation='relu',name='resnet_a_conv2_branch1')(model)
    branch1 = BatchNormalization()(branch1)
    branch1 = Conv2D(32, (3,3), padding='same', activation='relu',name='resnet_a_conv3_branch1')(branch1)
    branch1 = BatchNormalization()(branch1)
    branch2 = Conv2D(32, (1,1), padding='same', activation='relu',name='resnet_a_conv4_branch1')(model)
    branch2 = BatchNormalization()(branch2)
    branch2 = Conv2D(48, (3,3), padding='same', activation='relu',name='resnet_a_conv5_branch1')(branch2)
    branch2 = BatchNormalization()(branch2)
    branch2 = Conv2D(64, (3,3), padding='same', activation='relu',name='resnet_a_conv6_branch1')(branch2)
    branch2 = BatchNormalization()(branch2)
    x = concatenate([branch0, branch1,branch2], axis=-1)
    x = Conv2D(384, (1,1), padding='same', activation='relu',name='resnet_a_conv7')(x)
    x = BatchNormalization()(x)
    model = keras.layers.add([x, model])
    return model

### Reduction A

Os blocos reduction são formados em geral por três branches principais, variando-se a quantidade de filtros de cada branch. A estrutura, representada pela Figura abaixo, possui um branch inicial apenas com uma camada pooling de dimensão 3x3, enquanto o segundo branch possui uma camada convolucional de dimensão 3x3 e o último três camadas convolucionais. Ao final, as três branches são concatenadas e resultam na saída desse bloco.

![alt text](imgs/reduction.png "Title")



In [4]:
def reduction_a(model):
    branch0 = MaxPooling2D((3,3), strides=(2,2), padding='same',name='reduction_a')(model)
    branch1 = Conv2D(384, (3,3), strides=(2,2), padding='same', activation='relu')(model)
    branch1 = BatchNormalization()(branch1)
    branch2 = Conv2D(192, (1,1), padding='same', activation='relu')(model)
    branch2 = BatchNormalization()(branch2)
    branch2 = Conv2D(224, (3,3), padding='same', activation='relu')(branch2)
    branch2 = BatchNormalization()(branch2)
    branch2 = Conv2D(256, (3,3), strides=(2,2), padding='same', activation='relu')(branch2)
    branch2 = BatchNormalization()(branch2)
    model = concatenate([branch0,branch1,branch2],axis=-1)
    return model

### Resnet-B

De forma semelhante ao primeiro bloco ResNet, define-se duas branches e o resultado delas é utilizado como entrada de uma camada convolucional. Ao final, utilizamos o método add para formar a camada residual, utilizando-se como parâmetros o resultado da camada anterior (reduction-A) e o resultado dessa camada (a concatenação dos três branches).
![alt text](imgs/resnet-b.png "Title")



In [5]:
def inception_resnet_b(model):
    branch0 =  Conv2D(128, (1,1), padding='same', activation='relu', name='resnet_b')(model)
    branch0 = BatchNormalization()(branch0)
    branch0 =  Conv2D(160, (1,7), padding='same', activation='relu')(branch0)
    branch0 = BatchNormalization()(branch0)
    branch0 =  Conv2D(192, (7,1), padding='same', activation='relu')(branch0)
    branch0 = BatchNormalization()(branch0)   
    
    branch1 =  Conv2D(192, (1,1), padding='same', activation='relu')(model)
    branch1 = BatchNormalization()(branch1)
    x = concatenate([branch1, branch0], axis=-1)
    branch1 =  Conv2D(1024, (1,1), padding='same', activation='relu')(x)
    branch1 = BatchNormalization()(branch1)
    model = keras.layers.add([branch1, model])
    return model

### Reduction B

Para o segundo módulo reduction, utiliza-se quatro banches, uma apenas com uma camada pooling, dois com duas camadas convolucionais respectivamente e um último branch três camadas convolucionais. Após isso, os branches são concatenados e utilizados como saída do bloco.

![alt text](imgs/reduction-b.png "Title")



In [6]:
def reduction_b(model):
    branch0 = MaxPooling2D((3,3), strides=(2,2), padding='same', name='reduction_b')(model)
    branch1 = Conv2D(256, (1,1), padding='same', activation='relu')(model)
    branch1 = BatchNormalization()(branch1)
    branch1 = Conv2D(384, (3,3), padding='same', activation='relu')(branch1)
    branch1 = BatchNormalization()(branch1)
    branch2 = Conv2D(256, (1,1), padding='same', activation='relu')(model)
    branch2 = BatchNormalization()(branch2)
    branch2 = Conv2D(288, (3,3), padding='same', activation='relu')(branch2)
    branch2 = BatchNormalization()(branch2)
    branch3 = Conv2D(256, (1,1), padding='same', activation='relu')(model)
    branch3 = BatchNormalization()(branch3)
    branch3 = Conv2D(288, (3,3), padding='same', activation='relu')(branch3)
    branch3 = BatchNormalization()(branch3)
    branch3 = Conv2D(320, (1,1), padding='same', activation='relu')(branch3)
    branch3 = BatchNormalization()(branch3)
    model = concatenate([branch0,branch1,branch2,branch3], axis=-1)
    return model

### ResNet-C

Para o último bloco residual, cria-se duas branches com uma camada convolucional e três camadas convolucionais respectivamente. A saída das duas branches é concatenada e utilizada como entrada de uma camada convolucional e por fim, utiliza-se o método add do Keras para finalizar o bloco residual, utilizando-se como parâmetros o modelo de saída do bloco anterior (reduction) e o resultado desta camada (a concatenação dos três branches).

![alt text](imgs/resnet-c.png "Title")

In [7]:
def inception_resnet_c(model):
    branch0 =  Conv2D(192, (1,1), padding='same', activation='relu', name = 'resnet_c')(model)
    branch0 = BatchNormalization()(branch0)
    branch0 =  Conv2D(224, (1,3), padding='same', activation='relu')(branch0)
    branch0 = BatchNormalization()(branch0)
    branch0 =  Conv2D(256, (3,1), padding='same', activation='relu')(branch0)
    branch0 = BatchNormalization()(branch0)   
    
    branch1 =  Conv2D(192, (1,1), padding='same', activation='relu')(model)
    branch1 = BatchNormalization()(branch1)
    x = concatenate([branch1, branch0], axis=-1)
    branch1 =  Conv2D(2016, (1,1), padding='same', activation='relu')(x)
    branch1 = BatchNormalization()(branch1)
    model = keras.layers.add([branch1, model])
    return model

### Construindo uma arquitetura Inception

Após definir todos os blocos da arquitetura, pode-se criar o modelo final do Keras. Para isso, construimos o modelo seguindo a ordem da arquitetura principal da Figura 1. Após isso, adiciona-se uma camada Dropout e a camada Dense. Para finalizar, cria-se a instância Model do Keras passando-se como parâmetros a dimensão de entrada da imagem e o modelo final da camada Dense (out). Com isso o modelo é criado e pode ser treinado da mesma forma que qualquer arquitetura simples no Keras utilizando os métodos compile e fit.

In [8]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
from keras.layers import Input
input_img = Input(shape = (32, 32, 3))

In [9]:
from keras.models import Model

st = stem(input_img)
resnetA = inception_resnet_A(st)
reductionA = reduction_a(resnetA)
resnetB = inception_resnet_b(reductionA)
reductionB = reduction_b(resnetB)
resnetC = inception_resnet_c(reductionB)
#average_pool = AveragePooling2D()(resnetC)
drop = Dropout(0.8)(resnetC)
flatten = Flatten()(drop)
out = Dense(10, activation='softmax')(flatten)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

net = Model(inputs = input_img, outputs = out)
net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
net.fit(X_train, y_train, validation_split=(0.2), epochs=8, batch_size=32,callbacks=[tbCallBack])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 40000 samples, validate on 10000 samples
Epoch 1/8
40000/40000 [==============================] - 1086s - loss: 4.3234 - acc: 0.2611 - val_loss: 3.1176 - val_acc: 0.3252
Epoch 2/8
40000/40000 [==============================] - 940s - loss: 2.7996 - acc: 0.2793 - val_loss: 6.0659 - val_acc: 0.1759
Epoch 3/8
13184/40000 [========>.....................] - ETA: 805s - loss: 2.6662 - acc: 0.2689

KeyboardInterrupt: 